In [73]:
using Random
using DataStructures

In [74]:
# returns dup char --> num instances dictionary, num of duplicate chars  
##### returns 2 x N array (N = num dup char) of duplicate char & num instances, num of duplicate chars

function find_dups_in_str(str::String)
    alphabet = Set{Char}() 
    duplicates = OrderedDict{Char, Int}()  # duplicate char --> num instances 

    for char in str   
        if char in alphabet 
            if char in keys(duplicates)
                duplicates[char] += 1
            else 
                duplicates[char] = 2 
            end 
        else 
            push!(alphabet, char)
        end 
    end 

    return duplicates, length(duplicates) 
end 

find_dups_in_str (generic function with 1 method)

In [75]:
function generate_map(num_dups::Int, dup_to_num_instances::OrderedDict{Char, Int})
    map = Array{Int}(undef, num_dups)
    
    idx = 1
    # for each position of the map (each gene w >1 instances)
    for (dup_char, num_instances) in pairs(dup_to_num_instances)
        # an integer value is selcted uniformly from the interval [0, occ(α, S)! -1] 
        mapidx_for_dup_char = rand(0:factorial(num_instances)-1)
        
        map[idx] = mapidx_for_dup_char
        idx += 1
    end 
    
    return map
end 

generate_map (generic function with 2 methods)

In [81]:
# mapping S --> T, generates r random maps (RM)

function randommap(S::String, P::String, num_maps::Int)
    println("SRC ", S, " --> TARGET ", P, "\n")

    S_dup_to_num_instances, S_num_dups = find_dups_in_str(S)
    P_dup_to_num_instances, P_num_dups = find_dups_in_str(P)
    
    printstyled("source\n", color=:magenta)
    println("\ndup genes --> multiplicity")
    println(S_dup_to_num_instances, "\n")

    S_M = Array{Array{Int}}(undef, num_maps)  # does copy elements or point to them? 

    # r RM of src str S are generated and stored in a set S_M 
    println("source maps")
    for i in 1:num_maps
        s_map = generate_map(S_num_dups, S_dup_to_num_instances)
        println(s_map)
        push!(S_M, s_map)
    end 
        
    printstyled("\ntarget", color=:magenta)
    println("\ndup genes --> multiplicity")
    println(P_dup_to_num_instances, "\n")

    # choose an arbitrary map p for target string 
    P_map = generate_map(P_num_dups, P_dup_to_num_instances)
    println("target map")
    print(P_map, "\n")

    # lehmer code to convert S/P --> mapped S/P for each map 
    # convert int n --> nth permutation of n ints in lexicographical order 

   
        
    # intiially the target string is mapped into permutation Pp using an arbitrary map p 
    
    # for each map m in M 
        # distance between permutation Sm and Pp is computed using estimator algo 

    #return shortest map iwth shortest dist between permutations 
end 

src = "aaabbcdeffffgh"
target = "cbaaaa"
num_maps = 10

randommap(src, target, num_maps)

SRC aaabbcdeffffgh --> TARGET cbaaaa

source

dup genes --> multiplicity
OrderedDict('a' => 3, 'b' => 2, 'f' => 4)

source maps
[3, 1, 13]
[5, 1, 11]
[3, 0, 7]
[4, 0, 19]
[2, 0, 18]
[4, 0, 5]
[4, 1, 10]
[4, 1, 7]
[4, 0, 17]
[1, 1, 11]

target
dup genes --> multiplicity
OrderedDict('a' => 4)

target map
[16]
